#### Importar Librerias

In [1]:
import ray
from ray import serve
import mlflow
import pandas as pd
from fastapi import FastAPI
from fastapi import Request
from pydantic import BaseModel
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse
from starlette.requests import Request
from json import JSONDecodeError

#### Conecta a un clúster de Ray ya iniciado

In [11]:
try:
    #ray.init(address="ray://localhost:10001")
    
    #ray_head = "127.0.0.1"
    #ray.init(address=f'ray://{ray_head}:10001') #SERVER
    
    ray.init()
    
    serve.start(detached=True)
    print("Conexion correcta al servicio Ray ...")
except Exception as ex:
    print(f"Ray: Se produjo el siguiente error : {ex} ")

2025-05-26 20:13:39,048	ERROR services.py:1195 -- Failed to start the dashboard: Ray Client is not connected. Please connect by calling `ray.init`.
2025-05-26 20:13:39,049	ERROR services.py:1196 -- Ray Client is not connected. Please connect by calling `ray.init`.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/ray/_private/services.py", line 1140, in start_api_server
    dashboard_url = ray.experimental.internal_kv._internal_kv_get(
  File "/opt/conda/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return getattr(ray, func.__name__)(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/ray/util/client/__init__.py", line 276, in __getattr__
    return self.get_context().__getattr__(name)
  File "/opt/conda/lib/python3.8/site-packages/ray/util/client/__init__.py", line 177, in __getattr__
    raise Exception(
Exception: Ray Client is not connected. Please connect by calling `ray.init`.
2025-05-26 20:13:3

Ray: Se produjo el siguiente error :  


#### Configuracion del Modelo

In [3]:
try:
    app = FastAPI(
        title = "Import Predictor API",
        description = "Import Pipeline online inference"
    )

    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        mensajes = [err["msg"] for err in exc.errors()]
        return JSONResponse(
            status_code = 422,
            content = { "cod": 422, "msg": "; ".join(mensajes), "data": None, "errors": exc.errors(), },
        )
    
    class Item(BaseModel):
        "sepal length (cm)" : float
        "sepal width (cm)"  : float
        "petal length (cm)" : float
        "petal width (cm)"  : float
    
    @serve.deployment(
        route_prefix="/v1/gestionriesgo/evaluacionriesgoaduanero/contrab-carga-iamc/score/e/dtscore",
        ray_actor_options={"num_cpus": 0.1, "num_gpus": 0},
        name = "contrab-carga-iamc"
    )
    @serve.ingress(app)
    class ModelMlFlow:
        def __init__(self):
            mlflow.set_tracking_uri("http://172.26.56.133:5000")
            model_uri = "models:/Iris_LogReg_Model/Production"
            try:
                self.model = mlflow.sklearn.load_model(model_uri)
            except Exception as ex:
                print(f"MlFlow: Se produjo el siguiente error : {ex} ")

        @app.post("/")
        async def score(self, item: Item):
            try:
                #payload = await request.json()
                payload = item.dict()
                df = pd.DataFrame([payload])
                score = float(self.model.predict_proba(df)[0][1])
                return {"cod": 200, "msg": "Score calculated successfully.", "data": {"em_eventprobability": score}, "errors": None }
            except Exception as e:
                #logger.error(f"Error in import prediction: {e}")
                return {"cod": 500, "msg": "Error calculating score", "data": None, "errors": str(e)}
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex} ")

SyntaxError: illegal target for annotation (115353257.py, line 16)

#### Desplegar el Modelo

In [ ]:
try:
    ModelMlFlow.deploy()
    print("Modelo Desplegado")
except Exception as ex:
    serve.get_deployment("contrab-carga-iamc").delete()
    print(f"Ray: Se produjo el siguiente error : {ex} ")

http://172.26.56.134:8000/v1/gestionriesgo/evaluacionriesgoaduanero/contrab-carga-iamc/score/e/dtscore/docs

In [ ]:
serve.list_deployments()

In [ ]:
#serve.get_deployment("contrab-carga-iamc").delete()